<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/FractionsTest3/3_14_fraksi3_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook ini dijalankan menggunakan email shintafiaa@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)
    * [8.2. Gambaran BackTesting](#7.2)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


In [1]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-qzmxdrol
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-qzmxdrol
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-iirq79yc/pyfolio_79bc5ef5a8cd4af88b65630696470aa0
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-iirq79yc/pyfolio_79bc5ef5a8cd4af88b65630696470aa0
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-iirq79yc/elegantrl_aac9096580f7432aa738dcc9036cbec0
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-iirq79yc/elegantrl_aac9096580f7432aa738dcc9036cbec0
     |████████████████████████████████| 2.3 MB 12.8 MB/s 
     |████████████████████████████████| 234 kB 58.7 MB/s 
     |█████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

In [4]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [5]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: x<Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00<= x <Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00<= x <Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00<= x <Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >=Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']

In [108]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi3).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10916, 8)


In [109]:
df.nunique()

date       2729
open       1657
high       1700
low        1688
close      3584
volume    10043
tic           4
day           5
dtype: int64

In [110]:
print(df['tic'].unique())

['AKRA.JK' 'BRPT.JK' 'KLBF.JK' 'MEDC.JK']


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan Ascending atau Descending dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [111]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,138.000000,144.000000,130.000000,82.180923,2990000,AKRA.JK,0
1,2009-01-05,61.000000,70.000000,60.000000,64.386543,95470000,BRPT.JK,0
2,2009-01-05,81.000000,84.000000,80.000000,63.608833,165165000,KLBF.JK,0
3,2009-01-05,371.428619,382.857178,361.904785,349.914215,20,MEDC.JK,0
4,2009-01-06,126.000000,138.000000,126.000000,80.989868,2915000,AKRA.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.

In [112]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [113]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [114]:
processed.nunique()

date           2652
open           1645
high           1686
low            1682
close          3546
volume         9784
tic               4
day               5
macd          10602
rsi_30         9117
cci_30        10604
vix            1441
turbulence     2398
dtype: int64

In [115]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['AKRA.JK', 'BRPT.JK', 'KLBF.JK', 'MEDC.JK']

In [116]:
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

In [117]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'AKRA.JK'),
 ('2009-01-05', 'BRPT.JK'),
 ('2009-01-05', 'KLBF.JK'),
 ('2009-01-05', 'MEDC.JK'),
 ('2009-01-06', 'AKRA.JK'),
 ('2009-01-06', 'BRPT.JK'),
 ('2009-01-06', 'KLBF.JK'),
 ('2009-01-06', 'MEDC.JK'),
 ('2009-01-07', 'AKRA.JK'),
 ('2009-01-07', 'BRPT.JK'),
 ('2009-01-07', 'KLBF.JK'),
 ('2009-01-07', 'MEDC.JK'),
 ('2009-01-08', 'AKRA.JK'),
 ('2009-01-08', 'BRPT.JK'),
 ('2009-01-08', 'KLBF.JK'),
 ('2009-01-08', 'MEDC.JK'),
 ('2009-01-09', 'AKRA.JK'),
 ('2009-01-09', 'BRPT.JK'),
 ('2009-01-09', 'KLBF.JK'),
 ('2009-01-09', 'MEDC.JK'),
 ('2009-01-10', 'AKRA.JK'),
 ('2009-01-10', 'BRPT.JK'),
 ('2009-01-10', 'KLBF.JK'),
 ('2009-01-10', 'MEDC.JK'),
 ('2009-01-11', 'AKRA.JK'),
 ('2009-01-11', 'BRPT.JK'),
 ('2009-01-11', 'KLBF.JK'),
 ('2009-01-11', 'MEDC.JK'),
 ('2009-01-12', 'AKRA.JK'),
 ('2009-01-12', 'BRPT.JK'),
 ('2009-01-12', 'KLBF.JK'),
 ('2009-01-12', 'MEDC.JK'),
 ('2009-01-13', 'AKRA.JK'),
 ('2009-01-13', 'BRPT.JK'),
 ('2009-01-13', 'KLBF.JK'),
 ('2009-01-13', 'MED

In [118]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180923,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914215,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989868,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745906
15996,2019-12-18,AKRA.JK,774.000000,778.000000,766.000000,693.404358,66851500.0,2.0,6.737230,52.879121,100.001008,12.580000,1.839521
15997,2019-12-18,BRPT.JK,1445.000000,1480.000000,1435.000000,1469.766479,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839521
15998,2019-12-18,KLBF.JK,1635.000000,1635.000000,1610.000000,1541.399292,42302800.0,2.0,11.584219,55.676842,166.121455,12.580000,1.839521


In [119]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180923,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914215,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989868,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745906
15996,2019-12-18,AKRA.JK,774.000000,778.000000,766.000000,693.404358,66851500.0,2.0,6.737230,52.879121,100.001008,12.580000,1.839521
15997,2019-12-18,BRPT.JK,1445.000000,1480.000000,1435.000000,1469.766479,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839521
15998,2019-12-18,KLBF.JK,1635.000000,1635.000000,1610.000000,1541.399292,42302800.0,2.0,11.584219,55.676842,166.121455,12.580000,1.839521


In [120]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180923,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914215,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989868,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745906
15996,2019-12-18,AKRA.JK,774.000000,778.000000,766.000000,693.404358,66851500.0,2.0,6.737230,52.879121,100.001008,12.580000,1.839521
15997,2019-12-18,BRPT.JK,1445.000000,1480.000000,1435.000000,1469.766479,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839521
15998,2019-12-18,KLBF.JK,1635.000000,1635.000000,1610.000000,1541.399292,42302800.0,2.0,11.584219,55.676842,166.121455,12.580000,1.839521


In [121]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180923,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914215,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989868,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745906
15996,2019-12-18,AKRA.JK,774.000000,778.000000,766.000000,693.404358,66851500.0,2.0,6.737230,52.879121,100.001008,12.580000,1.839521
15997,2019-12-18,BRPT.JK,1445.000000,1480.000000,1435.000000,1469.766479,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839521
15998,2019-12-18,KLBF.JK,1635.000000,1635.000000,1610.000000,1541.399292,42302800.0,2.0,11.584219,55.676842,166.121455,12.580000,1.839521


In [122]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180923,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.0
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.0
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.0
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914215,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.0
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989868,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.0
5,2009-01-06,BRPT.JK,66.000000,71.000000,62.000000,60.484322,72175000.0,1.0,-0.087550,0.000000,-66.666667,38.560001,0.0
6,2009-01-06,KLBF.JK,81.000000,82.000000,80.000000,63.608833,44137500.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.0
7,2009-01-06,MEDC.JK,373.333374,380.952423,361.904785,342.736542,24719622.0,1.0,-0.161038,0.000000,-66.666667,38.560001,0.0
8,2009-01-07,AKRA.JK,136.000000,142.000000,130.000000,80.989868,12010000.0,2.0,-0.034050,0.000000,25.058261,43.389999,0.0
9,2009-01-07,BRPT.JK,63.000000,70.000000,61.000000,61.459877,80160000.0,2.0,-0.081304,20.547945,-100.000000,43.389999,0.0


In [123]:
processed_full.nunique()

date           2652
tic               4
open           1645
high           1686
low            1682
close          3546
volume         9784
day               5
macd          10602
rsi_30         9117
cci_30        10604
vix            1441
turbulence     2398
dtype: int64

In [124]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,13800.000000,14400.000000,13000.000000,8218.092346,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,6100.000000,7000.000000,6000.000000,6438.654327,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,8100.000000,8400.000000,8000.000000,6360.883331,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,37142.861938,38285.717773,36190.478516,34991.421509,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,12600.000000,13800.000000,12600.000000,8098.986816,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,80000.000000,83386.242676,75767.193604,72177.276611,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745906
15996,2019-12-18,AKRA.JK,77400.000000,77800.000000,76600.000000,69340.435791,66851500.0,2.0,6.737230,52.879121,100.001008,12.580000,1.839521
15997,2019-12-18,BRPT.JK,144500.000000,148000.000000,143500.000000,146976.647949,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839521
15998,2019-12-18,KLBF.JK,163500.000000,163500.000000,161000.000000,154139.929199,42302800.0,2.0,11.584219,55.676842,166.121455,12.580000,1.839521


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

In [125]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1980


In [126]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,MEDC.JK,76190.478516,77037.036133,74074.072266,70980.963135,57711622.0,3.0,17.428146,59.931074,74.238054,10.18,1.060702
2156,2017-12-29,AKRA.JK,129000.000000,129000.000000,127000.000000,103281.591797,17576000.0,4.0,-13.380603,45.158706,10.628049,11.04,0.497395
2156,2017-12-29,BRPT.JK,45200.000000,45800.000000,44200.000000,44095.022583,141907000.0,4.0,2.984994,53.936637,-35.182441,11.04,0.497395
2156,2017-12-29,KLBF.JK,167000.000000,170000.000000,167000.000000,153249.182129,40084400.0,4.0,9.240525,54.156937,136.738874,11.04,0.497395
2156,2017-12-29,MEDC.JK,75343.914795,76190.478516,73650.793457,70980.963135,38584586.0,4.0,16.804936,59.931074,56.067394,11.04,0.497395


In [127]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,AKRA.JK,127500.000000,128500.000000,123500.000000,100841.888428,25448000.0,1.0,-12.741419,42.797338,-27.402587,9.77,4.977822
0,2018-01-02,BRPT.JK,45400.000000,45400.000000,44000.000000,43899.908447,110809000.0,1.0,1.533833,53.320547,-60.693808,9.77,4.977822
0,2018-01-02,KLBF.JK,169000.000000,173000.000000,167500.000000,156876.367188,17137900.0,1.0,13.548819,58.402802,189.233495,9.77,4.977822
0,2018-01-02,MEDC.JK,75343.914795,77037.036133,75343.914795,71379.736328,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977822
1,2018-01-03,AKRA.JK,126000.000000,126500.000000,123000.000000,100435.266113,34013000.0,2.0,-13.495443,42.414937,-48.046627,9.15,6.195110


In [128]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


In [129]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [130]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.

In [131]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [132]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [133]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

--------------------------------------
| time/                 |            |
|    fps                | 373        |
|    iterations         | 100        |
|    time_elapsed       | 40         |
|    total_timesteps    | 15000      |
| train/                |            |
|    entropy_loss       | -5.73      |
|    explained_variance | 5.07e-06   |
|    learning_rate      | 0.0014     |
|    n_updates          | 99         |
|    policy_loss        | 2.74e+04   |
|    reward             | -938.83167 |
|    std                | 1.01       |
|    value_loss         | 6.61e+07   |
--------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 482406780.89
total_reward: 382406780.89
total_cost: 119439551.34
total_trades: 8108
Sharpe: 0.746
-------------------------------------
| time/                 |           |
|    fps                | 375       |
|    iterations         | 200       |
|    time_elapsed       | 79        |
|    total_times

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [134]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [135]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 105664198.69
total_reward: 5664198.69
total_cost: 92229173.52
total_trades: 8170
Sharpe: 0.129
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 118998034.31
total_reward: 18998034.31
total_cost: 93960528.42
total_trades: 8224
Sharpe: 0.202
-----------------------------------
| time/              |            |
|    fps             | 379        |
|    iterations      | 1          |
|    time_elapsed    | 131        |
|    total_timesteps | 50000      |
| train/             |            |
|    reward          | -48.391254 |
-----------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [136]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [137]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 1028936246.71
total_reward: 928936246.71
total_cost: 1658738.96
total_trades: 2673
Sharpe: 0.932
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 98       |
|    time_elapsed    | 87       |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 5.58e+08 |
|    critic_loss     | 1.12e+19 |
|    learning_rate   | 0.9      |
|    n_updates       | 6471     |
|    reward          | 0.0      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 86       |
|    time_elapsed    | 199      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 5.82e+07 |
|    critic_loss     | 9.59e+17 |
|    learning_rate   | 0.9      |
|    n_updates       | 15099    |
|    reward    

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [138]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [139]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 68        |
|    time_elapsed    | 125       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 5.38e+03  |
|    critic_loss     | 1.52e+05  |
|    ent_coef        | 1.25      |
|    learning_rate   | 0.0001    |
|    n_updates       | 8613      |
|    reward          | -267.0302 |
----------------------------------
day: 2156, episode: 80
begin_total_asset: 100000000.00
end_total_asset: 382244342.14
total_reward: 282244342.14
total_cost: 229465.75
total_trades: 6446
Sharpe: 0.632
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 68        |
|    time_elapsed    | 252       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 5.48e+03  |
|    critic_loss     | 6.58e+04  |
|    ent_coef        | 1.25 

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [140]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [141]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 461662463.49
total_reward: 361662463.49
total_cost: 229472.10
total_trades: 6468
Sharpe: 0.732
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 130       |
|    time_elapsed    | 66        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 2.9e+04   |
|    critic_loss     | 1.77e+08  |
|    learning_rate   | 0.0003    |
|    n_updates       | 6471      |
|    reward          | 77.076904 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 115       |
|    time_elapsed    | 149       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 2.92e+03  |
|    critic_loss     | 8.37e+06  |
|    learning_rate   | 0.0003    |
|    n_updates       | 15099

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp100.000.000.

In [142]:
data_risk_indicator = processed_full_lot[(processed_full_lot.date<TRAIN_END_DATE) & (processed_full_lot.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [143]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [144]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [145]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        3.808146
std         7.015115
min         0.000000
25%         0.656760
50%         1.918150
75%         4.171931
max       149.008706
Name: turbulence, dtype: float64

In [146]:
insample_risk_indicator.turbulence.quantile(0.996)

36.23675114477346

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [147]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [148]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [149]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [150]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [151]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [152]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [153]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return         -0.145327
Cumulative returns    -0.265425
Annual volatility      0.298924
Sharpe ratio          -0.377189
Calmar ratio          -0.365114
Stability              0.328277
Max drawdown          -0.398031
Omega ratio            0.933164
Sortino ratio         -0.533533
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.951218
Daily value at risk   -0.038108
dtype: float64


###8.1.2 Agen ppo

In [154]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return          0.786983
Cumulative returns     2.127783
Annual volatility      0.418587
Sharpe ratio           1.595148
Calmar ratio           2.067604
Stability              0.644740
Max drawdown          -0.380626
Omega ratio            1.358303
Sortino ratio          2.827729
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.414334
Daily value at risk   -0.050087
dtype: float64


###8.1.3 Agen TD3

In [155]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return         -0.085724
Cumulative returns    -0.161420
Annual volatility      0.496233
Sharpe ratio           0.062026
Calmar ratio          -0.141383
Stability              0.683353
Max drawdown          -0.606323
Omega ratio            1.011410
Sortino ratio          0.100910
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.303647
Daily value at risk   -0.062397
dtype: float64


###8.1.4 Agen SAC

In [156]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return         -0.107180
Cumulative returns    -0.199638
Annual volatility      0.344580
Sharpe ratio          -0.158433
Calmar ratio          -0.209621
Stability              0.549321
Max drawdown          -0.511303
Omega ratio            0.972205
Sortino ratio         -0.236695
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.133429
Daily value at risk   -0.043630
dtype: float64


###8.1.5 Agen DDPG

In [157]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return          0.141374
Cumulative returns     0.296596
Annual volatility      0.276673
Sharpe ratio           0.617036
Calmar ratio           0.378018
Stability              0.062416
Max drawdown          -0.373987
Omega ratio            1.114659
Sortino ratio          0.904545
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.053539
Daily value at risk   -0.034180
dtype: float64


###8.1.6. Baseline Stats

In [158]:
#baseline stats
print("===========Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^JKII", 
        start = df_account_value_a2c.loc[0,'date'],
        end = df_account_value_a2c.loc[len(df_account_value_a2c)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

===========Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (478, 8)
Annual return         -0.041972
Cumulative returns    -0.078113
Annual volatility      0.185511
Sharpe ratio          -0.138819
Calmar ratio          -0.175216
Stability              0.093175
Max drawdown          -0.239543
Omega ratio            0.977818
Sortino ratio         -0.189296
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.900451
Daily value at risk   -0.023474
dtype: float64
